In [1]:
## import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, Embedding
import tensorflow as tf

Using TensorFlow backend.
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lab606a/.virtualenvs

In [2]:
# set GPU memory
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
datasetset = pd.read_csv("./datasets/30balls/dataset20200331_30balls.csv", header=None)
datasetset = datasetset.fillna(0)
datasetset = np.array(datasetset)
datasetset.shape

(4000, 91)

In [4]:
#datasetset = sequence.pad_sequences(datasetset, maxlen=91, padding='post', dtype='float32')

In [5]:
#datasetset = datasetset[:,:31]
#datasetset.shape

In [6]:
x_train = datasetset[:,1:]
x_train.shape

(4000, 90)

In [7]:
row = int(x_train.shape[1]/3)
col = 3

In [8]:
x_train = x_train.reshape(x_train.shape[0], row, col)
x_train = x_train.astype('float32')
x_train.shape

(4000, 30, 3)

In [9]:
y_train = datasetset[:,0]
y_train = y_train.astype('int')
y_train.shape

(4000,)

In [10]:
n_classes = 8

In [11]:
y_train = np_utils.to_categorical(y_train, n_classes)
y_train.shape

(4000, 8)

In [12]:
model = Sequential()
model.add(LSTM(units=64, activation='tanh', input_shape=(row,col) , unroll=True, return_sequences=True))
#model.add(LSTM(units=256, activation='tanh', unroll=True, return_sequences=True))
#model.add(LSTM(units=128, activation='tanh', unroll=True, return_sequences=True))
#model.add(LSTM(units=64, activation='tanh', unroll=True, return_sequences=True, dropout=0.2))
model.add(LSTM(units=32, activation='tanh', unroll=True, return_sequences=True, dropout=0.2))
model.add(LSTM(units=16, activation='tanh', unroll=True, return_sequences=True, dropout=0.2))
model.add(LSTM(units=8, activation='tanh', unroll=True, dropout=0.2))
model.add(Dense(units=n_classes, activation='softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 64)            17408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 32)            12416     
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 16)            3136      
_________________________________________________________________
lstm_4 (LSTM)                (None, 8)                 800       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
Total params: 33,832
Trainable params: 33,832
Non-trainable params: 0
_________________________________________________________________


In [62]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [78]:
# training
batch_size = 4000
training_iters = 50
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=training_iters, shuffle=True)

Epoch 1/50
4000/4000 [==============================] - 0s 33us/step - loss: 0.0342 - acc: 0.9908
Epoch 2/50
4000/4000 [==============================] - 0s 33us/step - loss: 0.0199 - acc: 0.9955
Epoch 3/50
4000/4000 [==============================] - 0s 29us/step - loss: 0.0323 - acc: 0.9927
Epoch 4/50
4000/4000 [==============================] - 0s 28us/step - loss: 0.0346 - acc: 0.9902
Epoch 5/50
4000/4000 [==============================] - 0s 28us/step - loss: 0.0267 - acc: 0.9930
Epoch 6/50
4000/4000 [==============================] - 0s 28us/step - loss: 0.0321 - acc: 0.9920
Epoch 7/50
4000/4000 [==============================] - 0s 29us/step - loss: 0.0290 - acc: 0.9920
Epoch 8/50
4000/4000 [==============================] - 0s 28us/step - loss: 0.0289 - acc: 0.9930
Epoch 9/50
4000/4000 [==============================] - 0s 28us/step - loss: 0.0214 - acc: 0.9952
Epoch 10/50
4000/4000 [==============================] - 0s 28us/step - loss: 0.0427 - acc: 0.9893
Epoch 11/50
4000/40

In [51]:
def cal_acc(direction, speed):
    #num1 = num
    #num = int(num)
    speed = str(speed)
    file = './datasets/30balls/' +  direction + speed + '_test.csv'
    #file = './datasets/top5_test.csv'
    test_up2 = pd.read_csv(file, header=None)
    test_up2 = test_up2.fillna(0)
    test_up2 = np.array(test_up2)
    #test_up2 = test_up2[:,:30]
    #print(test_up2.shape)
    test_up2 = test_up2.reshape(test_up2.shape[0],row,col)
    cnt = np.array([0,0,0,0,0,0,0,0])
    pred = model.predict(test_up2)
    for i in range (pred.shape[0]):
        for j in range (8):
            if max(pred[i,:]) == pred[i,j]:
                cnt[j] += 1
            #else:
                #print(i)
        #print("i = ",i+1)
        #print("\n",cnt)
    #print('------------------------------')
    return cnt

In [79]:
accuarcy = np.zeros([8,8])
accuarcy[0,:] = cal_acc('top', 5)
accuarcy[1,:] = cal_acc('top', 6)
accuarcy[2,:] = cal_acc('left', 5)
accuarcy[3,:] = cal_acc('left', 6)
accuarcy[4,:] = cal_acc('right', 5)
accuarcy[5,:] = cal_acc('right', 6)
accuarcy[6,:] = cal_acc('back', 5)
accuarcy[7,:] = cal_acc('back', 6)
accuarcy

array([[243.,   6.,   0.,   0.,   0.,   0.,   1.,   0.],
       [  3., 247.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0., 249.,   1.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   2., 248.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0., 248.,   2.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   1., 249.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0., 250.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0., 250.]])

In [73]:
#model.save('./classification_30ball_20200421_64to8')

In [46]:
top5_test = pd.read_csv('./datasets/30balls/left6_test.csv', header=None)
top5_test = np.array(top5_test)
top5_test.shape

(250, 90)

In [47]:
#tmp = top5_test[0,:]

np.set_printoptions(suppress=True)
for i in range(14,90,3):
    tmp = top5_test[0,:i]
    tmp = tmp.reshape(1, tmp.shape[0])
    tmp = sequence.pad_sequences(tmp, maxlen=(row*col), padding='post', dtype='float32')
    tmp = tmp.reshape(1, 30, 3)
    print((i+1)/3)
    preddd = model.predict(tmp, verbose=1)
    print(preddd)
    #print(tmp.shape)


5.0
1/1 [==============================] - 0s 18ms/step
[[0.01006124 0.00007977 0.00097058 0.00327141 0.9766767  0.00199585
  0.0067256  0.00021875]]
6.0
1/1 [==============================] - 0s 19ms/step
[[0.00890218 0.00006573 0.00077546 0.00293975 0.979228   0.00175653
  0.00616225 0.00017005]]
7.0
1/1 [==============================] - 0s 17ms/step
[[0.00861334 0.00007399 0.00070117 0.00300402 0.9793504  0.00202647
  0.0060531  0.00017755]]
8.0
1/1 [==============================] - 0s 16ms/step
[[0.00796636 0.00017281 0.00063479 0.00402901 0.97482544 0.00549119
  0.00655366 0.00032663]]
9.0
1/1 [==============================] - 0s 18ms/step
[[0.00099517 0.00736791 0.00006448 0.01607219 0.15574145 0.8100198
  0.00499249 0.00474643]]
10.0
1/1 [==============================] - 0s 18ms/step
[[0.00061519 0.00719655 0.00003838 0.01276407 0.09956695 0.8723896
  0.00360734 0.0038219 ]]
11.0
1/1 [==============================] - 0s 16ms/step
[[0.00054414 0.00693284 0.00003324 0.0123869